

# Generator 

In [3]:
#!/usr/bin/env python3
import sys
import numpy as np
from keras.layers import Dense, Reshape
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model


block_size = 128
num_blocks=4


generator = Sequential()
generator.add(Dense(block_size, input_shape=(100, )))
generator.add(LeakyReLU(alpha=0.02))
generator.add(BatchNormalization(momentum=0.8))

for i in range(num_blocks-1):
    block_size = block_size * 2
    generator.add(Dense(block_size))
    generator.add(LeakyReLU(alpha=0.2))
    generator.add(BatchNormalization(momentum=0.8))

generator.add(Dense(28 * 28 * 1, activation='tanh'))
generator.add(Reshape((28, 28, 1)))
generator.compile(loss="binary_crossentropy", optimizer='adam')

# Discriminator 

In [5]:
import sys
import numpy as np
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model


SHAPE =  (28, 28, 1)
CAPACITY = 28 * 28 * 1


discriminator = Sequential()
discriminator.add(Flatten(input_shape = SHAPE))
discriminator.add(Dense(CAPACITY, input_shape=SHAPE))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(int(CAPACITY/2)))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss="binary_crossentropy", optimizer='adam')

In [6]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 784)               615440    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 392)               307720    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 392)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 393       
Total params: 923,553
Trainable params: 923,553
Non-trainable params: 0
________________________________________________

# GAN

In [7]:
#GAN
discriminator.trainable = False

gan = Sequential()
gan.add(generator)
gan.add(discriminator)
gan.compile(loss="binary_crossentropy", optimizer='adam')

# load mnist


In [8]:
model_type = -1
def load_MNIST(model_type):

    allowed_types = [-1,0,1,2,3,4,5,6,7,8,9]

    if model_type not in allowed_types:
        print('ERROR: Only Integer Values from -1 to 9 are allowed')

    (X_train, Y_train), (_, _) = mnist.load_data()
    
    xxx = X_train
#     print("X_train: ", X_train.shape)
    if model_type!=-1:
        X_train = X_train[np.where(Y_train==int(model_type)) [0]]
        print("X_train: ", X_train.shape)

    X_train = ( np.float32(X_train) - 127.5) / 127.5
#     X_train = np.expand_dims(X_train, axis=3)
    
    return X_train, Y_train, xxx

In [9]:
X_train, Y_train, xxx = load_MNIST(model_type=3)


NameError: name 'mnist' is not defined

In [ ]:
img = plt.imshow(X_train[0])


In [ ]:
from random import randint

# Grab a batch
count_real_images = int(32/2)
starting_index = randint(0, (len(X_train) - count_real_images ))
starting_index

In [ ]:
real_images_raw = X_train[starting_index : (starting_index + count_real_images)]
real_images_raw.shape

In [ ]:
img = plt.imshow(real_images_raw[0])


In [ ]:
x_real_images = real_images_raw.reshape(count_real_images, 28, 28, 1)
x_real_images.shape

In [ ]:
y_real_lebels = np.ones([count_real_images, 1])
y_real_lebels.shape

# Grab Generated Images for this training batch

In [ ]:
LATENT_SPACE_SIZE = 100
def sample_latent_space(instances):
        return np.random.normal(0, 1, (instances, LATENT_SPACE_SIZE))

In [ ]:
# Grab Generated Images for this training batch
latent_space_samples = sample_latent_space(count_real_images)
latent_space_samples.shape

In [ ]:

x_generated_images = generator.predict(latent_space_samples)

In [ ]:
y_generated_labels = np.zeros([32-count_real_images,1])
y_generated_labels.shape 

In [ ]:

print(y_generated_labels[0], y_real_lebels[0])

In [ ]:
# Combine to train on the discriminator
x_batch = np.concatenate( [x_real_images, x_generated_images] )
y_batch = np.concatenate( [y_real_lebels, y_generated_labels] )

In [ ]:
x_batch.shape

In [ ]:
y_batch[16]


# Train Adversarial Net

In [ ]:
# Now, train the discriminator with this batch
discriminator_loss = discriminator.train_on_batch(x_batch,y_batch)
print("ok")

In [ ]:
# Generate Noise
x_latent_space_samples = sample_latent_space(32)
y_generated_labels = np.ones([32,1])
y_generated_labels.shape

In [ ]:
generator_loss = gan.train_on_batch(x_latent_space_samples, y_generated_labels)


In [ ]:
print ('Epoch: '+str(int(1))+', [Discriminator :: Loss:'+str(discriminator_loss)+'], [ Generator :: Loss:'+str(generator_loss)+']')
